In [ ]:
## 计算同源基因对之间的P—values (T-test)
import pandas as pd
from scipy import stats
#df_hg = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/Ghir_all_homo_gene_pairs.txt', sep='\t', columns=['At', 'Dt'])
df_heg = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/all_hg_expression.txt', sep='\t')
pv_list = []
for i, r in df_heg.iterrows():
    rep1 = r[['FPKM_at_rep1', 'FPKM_at_rep2']].copy()
    rep2 = r[['FPKM_dt_rep1', 'FPKM_dt_rep2']].copy()
    p_value = stats.ttest_rel(rep1, rep2)
    pv_list.append(p_value[1])
    #df_heg.loc[i, 'pvalue'] = p_value
df_heg['pvalue'] = pv_list
df_heg['pvalue'].fillna(0, inplace=True)
df_heg.to_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_hg_expression_bias.txt', sep='\t', index=False, columns=['Geneid_at','Chr_at', 'Start_at', 'End_at', 'Geneid_dt', 'Chr_dt', 'Start_dt', 'End_dt', 'FPKM_at', 'FPKM_dt', 'pvalue'])

In [42]:
## 分析同源基因表达bias |Flod change|>=2 & pvalue<= 0.05
import pandas as pd
df = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_hg_expression_bias.txt', sep='\t')
df['FC'] = (df['FPKM_at']+0.001)/(df['FPKM_dt']+0.001)
df_bias = df.query('(FC>=2|FC<=0.5)&pvalue<=0.05').copy()
df_bias['Bias'] = df_bias['FC'].apply(lambda x: 'biasA' if x>=2 else 'biasD')
df_bias.to_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_bias_expression.txt', sep='\t', index=False)
df_tmp = df_bias.groupby('Bias', as_index=False)['Geneid_at'].count()
print(df_tmp)


    Bias  Geneid_at
0  biasA       1270
1  biasD       1616


In [16]:
## 所有差异表达基因 绘制loop 互作图
import pandas as pd
df = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/all_hg_expression.txt', sep='\t')
df['diff'] = (df['FPKM_at']+0.1)/(df['FPKM_dt']+0.1)
df_diff = df.query("(diff>=4 | diff<=0.25) & (FPKM_at>=1| FPKM_dt>=1) & Chr_at.str.contains('Ghir') & Chr_dt.str.contains('Ghir')", engine='python')
fp = open("F:/TM-1_leaf_3D-genome/RNA_Seq/diff_exp_gene/plot_diff_gene.sh", 'w')
for index, row in df_diff.iterrows():
    if row['diff'] >= 4:
        line_at = "sh plot_loop.sh {0} {1} {2} diff_gene/a_{3}_{4}_at.pdf".format(row['Chr_at'], row['Start_at']-200000, row['End_at']+200000, row['Geneid_at'], row['Geneid_dt'])
        line_dt = "sh plot_loop.sh {0} {1} {2} diff_gene/a_{3}_{4}_dt.pdf".format(row['Chr_dt'], row['Start_dt']-200000, row['End_dt']+200000, row['Geneid_at'], row['Geneid_dt'])
    else:
        line_at = "sh plot_loop.sh {0} {1} {2} diff_gene/d_{3}_{4}_at.pdf".format(row['Chr_at'], row['Start_at']-200000, row['End_at']+200000, row['Geneid_at'], row['Geneid_dt'])
        line_dt = "sh plot_loop.sh {0} {1} {2} diff_gene/d_{3}_{4}_dt.pdf".format(row['Chr_dt'], row['Start_dt']-200000, row['End_dt']+200000, row['Geneid_at'], row['Geneid_dt'])
    print(line_at, file=fp)
    print(line_dt, file=fp)
fp.close()


In [5]:
### 分析同源基因在TAD 上的分布差异
import pandas as pd
df_hg_position = pd.read_csv("F:/TM-1_leaf_3D-genome/RNA_Seq/all_homo_gene_position.txt", sep="\t", names=['c', 's', 'e', 'GeneId'])
df_TAD = pd.read_csv("F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/TM-1_TADLib_tad_20K_2M_choosed.bed", sep="\t", names=['c', 's', 'e', 'order'])
all_chrs = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
df_tmp = pd.DataFrame()
for c in all_chrs:
    df_hg_chr = df_hg_position.query("c==@c").copy()
    df_tad_chr = df_TAD.query("c==@c").copy()
    for index, row in df_tad_chr.iterrows():
        s, e = row['s'], row['e']
        start1, end1 = s, s+40000
        start2, end2 = e-40000, e
        start3, end3 = s+40000, e-40000
        #print(start1, end1, start2, end2, start3, end3)
        df_tmp1 = df_hg_chr.query("s<=@end1 & e>=@start1")[['c', 's', 'e', 'GeneId']].copy()
        df_tmp2 = df_hg_chr.query("s<=@end2 & e>=@start2")[['c', 's', 'e', 'GeneId']].copy()
        df_tmp3 = df_hg_chr.query("s<=@end3 & e>=@start3")[['c', 's', 'e', 'GeneId']].copy()
        #print(df_tmp1)
        #print(df_tmp2)
        #print(df_tmp3)
        df_tmp1['tags'] = ['boundary']*df_tmp1.shape[0]
        df_tmp2['tags'] = ['boundary']*df_tmp2.shape[0]
        df_tmp3['tags'] = ['center']*df_tmp3.shape[0]
        df_tmp = df_tmp.append([df_tmp1, df_tmp2, df_tmp3], ignore_index=True)
df_hg_pair = pd.read_csv("F:/TM-1_leaf_3D-genome/RNA_Seq/all_homo_gene_pairs_position.txt", sep="\t")
df_tmp.drop_duplicates(keep='first', subset=['c', 's', 'e', 'GeneId'], inplace=True)
df_tmp_at = df_tmp.query("c.str.contains('Ghir_A')", engine="python").copy()
df_tmp_dt = df_tmp.query("c.str.contains('Ghir_D')", engine="python").copy()
df_tmp_at.columns = ['chr_at', 'start_at', 'end_at', 'geneid_at', 'tags_at']
df_tmp_dt.columns = ['chr_dt', 'start_dt', 'end_dt', 'geneid_dt', 'tags_dt']
df_one = pd.merge(df_hg_pair, df_tmp_at, how='outer', on=['chr_at', 'start_at', 'end_at', 'geneid_at'])
df_result = pd.merge(df_one, df_tmp_dt, how='outer', on=['chr_dt', 'start_dt', 'end_dt', 'geneid_dt'])
df_result.to_csv("F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/gene_position/hg_TAD_position.txt", sep='\t', index=False)



In [3]:
df_result = pd.merge(df_one, df_tmp_dt, how='outer', on=['chr_dt', 'start_dt', 'end_dt', 'geneid_dt'])
df_result.to_csv("F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/gene_position/hg_TAD_position.txt", sep='\t', index=False)

In [6]:
df_result.dropna(inplace=True)
df_boundary_center = df_result.query("tags_at!=tags_dt")
print(df_boundary_center.shape[0])

9115


In [8]:
## 分析有多少差异表达基因 位于TAD boundary和TAD center 位置

df_gene_bias = pd.read_csv("F:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_bias_expression.txt", sep='\t')
df_gene_bias_id = df_gene_bias[['Geneid_at', 'Geneid_dt']]
df_gene_bias_id.columns = ['geneid_at', 'geneid_dt']
df_boundary_center_id = df_boundary_center[['geneid_at', 'geneid_dt']]
df_diff_TAD_diff_gene = pd.merge(df_boundary_center_id, df_gene_bias_id, how='inner')
print(df_diff_TAD_diff_gene.shape[0])

1088
